In [1]:
import pandas as pd
import numpy as np

titles = ['ADT' ,  'AHM' ,  'AoR' ,  'OE1O' , 'Anæstesidata' , 'Anæstesihændelse' ,  'Blødning' ,  'ca_vitale_værdier' ,   'Dialyse' ,  'EKG_data' ,  'Ekkokardiografi' ,  'Højde' ,  'intellispace' ,  'ITA' ,  'KAG_data' ,  'medicin' ,  'Pleuradræn' ,  'Populationen' ,  'Problemliste' , 'Respirator_data' ,  'Samlet_udskillelse' ,  'Spiromitri' ,  'Total_indgift' ,  'Urin' ,  'Vægt' ,  'viewpoint' ]

for title in titles:
    parquet_file = f'Processed_parquet_real\{title}.parquet.gzip'
    df = pd.read_parquet(parquet_file)
    globals()[title] = df

Spiromitri.drop(index=[2685,2686,2687,2688],inplace=True) # removes the rows with / in the values
Lab_svar = pd.read_csv(r'opdateret_data\Lab_svar.csv')
Lab_svar_1=Lab_svar[:2830372]
Lab_svar_2 = Lab_svar[2830372:]

Lab_svar_1 = Lab_svar_1.rename(columns={ 'V2' : 'What is being measured' ,  'V3' :  'Timestamp' ,  'V4' :  'Results' })
Lab_svar_1 = Lab_svar_1.dropna(subset=['Results'])
Lab_svar_1['Results'] = pd.to_numeric(Lab_svar_1['Results'], errors='coerce')
Lab_svar_1 = Lab_svar_1.groupby(['Timestamp','IDno','What is being measured']).Results.mean().reset_index()
Lab_svar_1 = Lab_svar_1.pivot(index = ['IDno','Timestamp'], columns=['What is being measured'],values='Results')
Lab_svar_1.reset_index(inplace=True)
Lab_svar_1['Timestamp'] = pd.to_datetime(Lab_svar_1['Timestamp'])

Lab_svar_2 = Lab_svar_2.rename(columns={ 'V2' : 'What is being measured' ,  'V3' :  'Timestamp' ,  'V4' :  'Results' })
Lab_svar_2 = Lab_svar_2.dropna(subset=['Results'])
Lab_svar_2['Results'] = pd.to_numeric(Lab_svar_2['Results'], errors='coerce')
Lab_svar_2 = Lab_svar_2.groupby(['Timestamp','IDno','What is being measured']).Results.mean().reset_index()
Lab_svar_2 = Lab_svar_2.pivot(index = ['IDno','Timestamp'], columns=['What is being measured'],values='Results')
Lab_svar_2.reset_index(inplace=True)
Lab_svar_2['Timestamp'] = pd.to_datetime(Lab_svar_2['Timestamp'])

Spiromitri=Spiromitri.rename(columns={"Resultatdato": "Timestamp"})

pre_liste = []

In [2]:
# Assuming Populationen and Anæstesihændelse are DataFrames
IDs_listx = Populationen['IDno'].unique()

no_induktion = []
no_stop = []
nothing = []
more_induktion = []
more_stop = []
missing_events = []
for ID in IDs_listx:
    # Filter the rows where ID matches and Hændelse is 'Induktion' or 'Stop Data Indsamling'
    event_rows_induk = Anæstesihændelse[(Anæstesihændelse['IDno'] == ID) & (Anæstesihændelse['Hændelse'] == 'Induktion')]
    event_rows_stop = Anæstesihændelse[(Anæstesihændelse['IDno'] == ID) & (Anæstesihændelse['Hændelse'] == 'Stop Data Indsamling')]
    event_rows_bypass_start = Anæstesihændelse[(Anæstesihændelse['IDno'] == ID) & (Anæstesihændelse['Hændelse'] == 'CV Bypass Start')]
    event_rows_aorta_tang_på = Anæstesihændelse[(Anæstesihændelse['IDno'] == ID) & (Anæstesihændelse['Hændelse'] == 'Aorta tang på')]
    event_rows_aorta_tang_af = Anæstesihændelse[(Anæstesihændelse['IDno'] == ID) & (Anæstesihændelse['Hændelse'] == 'Aorta tang af')]
    event_rows_bypass_slut = Anæstesihændelse[(Anæstesihændelse['IDno'] == ID) & (Anæstesihændelse['Hændelse'] == 'CV Bypass slut')]

    # Check if any event is missing
    if (event_rows_bypass_start.empty or event_rows_aorta_tang_på.empty or event_rows_aorta_tang_af.empty or event_rows_bypass_slut.empty):
        missing_events.append(ID)

    
    # Check for more than one row
    if  len(event_rows_induk) > 1:
        more_induktion.append(ID)
    if len(event_rows_stop) > 1:
        more_stop.append(ID)

    # If it has nothing
    if event_rows_induk.empty and event_rows_stop.empty:
        nothing.append(ID)
    
    # Add ID if it doesn't have either 'Induktion' or 'Stop Data Indsamling'
    if event_rows_induk.empty:
        if ID not in nothing:
            no_induktion.append(ID)
    if event_rows_stop.empty:
        if ID not in nothing:
            no_stop.append(ID)

In [3]:
dataframes = {
    'ADT': ADT,
    'AHM': AHM,
    'AoR': AoR,
    'OE1O': OE1O,
    'Anæstesidata': Anæstesidata,
    'Anæstesihændelse': Anæstesihændelse,
    'Blødning': Blødning,
    'ca_vitale_værdier': ca_vitale_værdier,
    'Dialyse': Dialyse,
    'EKG_data': EKG_data,
    'Ekkokardiografi': Ekkokardiografi,
    'Højde': Højde,
    'intellispace': intellispace,
    'ITA': ITA,
    'KAG_data': KAG_data,
    'Lab_svar': Lab_svar,
    'medicin': medicin,
    'Pleuradræn': Pleuradræn,
    'Populationen': Populationen,
    'Problemliste': Problemliste,
    'Respirator_data': Respirator_data,
    'Samlet_udskillelse': Samlet_udskillelse,
    'Spiromitri': Spiromitri,
    'Total_indgift': Total_indgift,
    'Urin': Urin,
    'Vægt': Vægt,
    'viewpoint': viewpoint
}

def check_timestamp_header_and_type(df_dict):
    results = []
    for name, df in df_dict.items():
        if 'Timestamp' in df.columns:
            timestamp_type = df['Timestamp'].dtype
            if timestamp_type == np.dtype('O'):
                try:
                    # Try parsing with the default format
                    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
                except Exception as e_default:
                    print(f"Default parsing failed for DataFrame '{name}' with error: {e_default}")
                    try:
                        # Try parsing with dayfirst=True
                        df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True, dayfirst=True)
                    except Exception as e_dayfirst:
                        print(f"Parsing with dayfirst=True failed for DataFrame '{name}' with error: {e_dayfirst}")
                        try:
                            # If parsing fails, you can specify a custom format or handle mixed formats
                            # Example: df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d-%m-%Y', utc=True)
                            df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True, errors='coerce')
                            if df['Timestamp'].isnull().any():
                                raise ValueError("Some dates could not be parsed and resulted in NaT values.")
                        except Exception as e_custom:
                            print(f"Custom parsing failed for DataFrame '{name}' with error: {e_custom}")
                            results.append((name, False, None))
                            continue
                timestamp_type = df['Timestamp'].dtype  # Update type after conversion
            results.append((name, True, timestamp_type))
        else:
            results.append((name, False, None))
    return results

# Check each DataFrame in the dictionary for the 'Timestamp' header and its type
results = check_timestamp_header_and_type(dataframes)

In [4]:
import pandas as pd
import numpy as np

# List of DataFrames
Dataframes_pre = [OE1O, Dialyse, EKG_data, Ekkokardiografi, Højde, KAG_data, Lab_svar_1, Lab_svar_2, Problemliste, Spiromitri, Vægt]

# List of DataFrame names
df_names = ['OE1O', 'Dialyse', 'EKG_data', 'Ekkokardiografi', 'Højde', 'KAG_data', 'Lab_svar_1', 'Lab_svar_2', 'Problemliste', 'Spiromitri', 'Vægt']

# Unique IDs from Populationen and filter them
IDs_listx = np.unique(Populationen['IDno'])
IDs_listx = [ID for ID in IDs_listx if ID not in no_induktion and ID not in no_stop and ID not in nothing and ID not in more_induktion and ID not in more_stop and ID not in missing_events]

# Initialize a dictionary to store filtered DataFrames
filtered_dataframes = {}

# Function to filter DataFrame by IDs and Timestamp
def filter_dataframe_by_ids_and_timestamp(IDs_listx, df, Anæstesihændelse):
    filtered_dfs = []

    for idno in IDs_listx:
        # Filter Anæstesihændelse based on ID number
        eeeeet = Anæstesihændelse[Anæstesihændelse['IDno'] == idno]
        if not eeeeet.empty:
            eeeeet_time = eeeeet[eeeeet['Hændelse'] == 'Induktion']['Timestamp'].values[0]
            eeeeet_time_utc = pd.Timestamp(eeeeet_time, tz='UTC')

            # Filter current DataFrame `df` based on the Timestamp column
            current_df = df[df['IDno'] == idno]
            filtered_population = current_df[current_df['Timestamp'] < eeeeet_time_utc]

            # Append the filtered DataFrame to the list
            filtered_dfs.append(filtered_population)

    # Concatenate all filtered DataFrames into one
    if filtered_dfs:
        combined_df = pd.concat(filtered_dfs, ignore_index=True)
    else:
        combined_df = pd.DataFrame(columns=df.columns)
    
    return combined_df

filtered_Population = filter_dataframe_by_ids_and_timestamp(IDs_listx, Populationen, Anæstesihændelse)
IDs_listx = filtered_Population['IDno'].unique()   

# Process each DataFrame in Dataframes_pre
for df_name, df in zip(df_names, Dataframes_pre):
    # Ensure the DataFrame has the 'IDno' column
    if 'IDno' in df.columns:
        # Apply the filtering function to each DataFrame
        filtered_df = filter_dataframe_by_ids_and_timestamp(IDs_listx, df, Anæstesihændelse)
        filtered_dataframes[df_name] = filtered_df
    else:
        print(f"DataFrame {df_name} does not contain 'IDno' column.")

# # Validate if the length of unique IDs in filtered_Population matches IDs_listx
# if 'Populationen' in filtered_dataframes:
#     unique_ids_in_filtered_population = filtered_dataframes['Populationen']['IDno'].nunique()
#     if unique_ids_in_filtered_population != len(IDs_listx):
#         print(f"Mismatch: {unique_ids_in_filtered_population} unique IDs in filtered_Population, expected {len(IDs_listx)}.")

# Extract the filtered DataFrames back to individual variables if needed
     

filtered_OE1O = filtered_dataframes.get('OE1O', None)
filtered_Dialyse = filtered_dataframes.get('Dialyse', None)
filtered_EKG_data = filtered_dataframes.get('EKG_data', None)
filtered_Ekkokardiografi = filtered_dataframes.get('Ekkokardiografi', None)
filtered_Højde = filtered_dataframes.get('Højde', None)
filtered_KAG_data = filtered_dataframes.get('KAG_data', None)
filtered_Lab_svar_1 = filtered_dataframes.get('Lab_svar_1', None)
filtered_Lab_svar_2 = filtered_dataframes.get('Lab_svar_2', None)
filtered_Problemliste = filtered_dataframes.get('Problemliste', None)
filtered_Spiromitri = filtered_dataframes.get('Spiromitri', None)
filtered_Vægt = filtered_dataframes.get('Vægt', None)

# Assuming AoR is another DataFrame you want to filter
filtered_AoR = AoR[AoR['IDno'].isin(IDs_listx)] 


In [5]:
len(filtered_Population['IDno'].unique())

3185

In [6]:
IDS_dfx = pd.DataFrame({'IDno': IDs_listx})
len(IDS_dfx['IDno'])

3185

In [7]:
list1 = list(IDS_dfx['IDno']).sort()
list2 = list(filtered_Population['IDno']).sort()

list1==list2

True

In [11]:
for id in list(filtered_Ekkokardiografi['IDno'].unique()):
    if id in IDs_listx:
        continue
    else:
        print(f'{id} is wrong')

Længden af ID liste skal være 3448 i forhold til IDS_listx som bliver til 3185 da vi fratager dem som har haft operationsdato efter induktion, dvs. dem der kun har Induktion og stop data indsamling (hvor vi antager at de derfor ikke har fået hjerteoperation, vi har fjernet ID'er, hvor der mangler faser under operationen)

Process Vægt

In [12]:
Processed_Vægt = filtered_Vægt.drop(columns = 'Timestamp')
weight_map = {
    'Kvinde': 71.4,
    'Mand': 86.4
}

merged = pd.merge(Processed_Vægt['IDno'], filtered_Population, on='IDno', how='right', indicator=True)

missing_ids = merged[merged['_merge'] == 'right_only']['IDno']

rows_to_append = []
for id_no in missing_ids:
    gender = filtered_Population.loc[filtered_Population['IDno'] == id_no, 'Køn'].iloc[0]
    weight_value = weight_map[gender]
    rows_to_append.append({'IDno': id_no, 'Vægt': weight_value})

# Append rows to Processed_Højde
if rows_to_append:
    Processed_Vægt = pd.concat([Processed_Vægt, pd.DataFrame(rows_to_append)], ignore_index=True)

Processed_Vægt['Vægt'] = pd.to_numeric(Processed_Vægt['Vægt'])
Processed_Vægt = Processed_Vægt.groupby('IDno')['Vægt'].mean().reset_index()
pre_liste.append(Processed_Vægt)

Process Problemliste

In [13]:
Processed_Problemliste = filtered_Problemliste.drop(columns = ['Timestamp', 'Diagnose navn'])
Processed_Problemliste['Diagnose SKS'] = Processed_Problemliste['Diagnose SKS'].str[:2]
Processed_Problemliste = Processed_Problemliste.drop_duplicates(subset=['Diagnose SKS', 'IDno'])
encoded_dia2 = pd.get_dummies(Processed_Problemliste['Diagnose SKS'])
Processed_Problemliste = pd.concat([Processed_Problemliste, encoded_dia2.astype(int)], axis=1)
Processed_Problemliste = Processed_Problemliste.drop(columns=['Diagnose SKS'])
missing_ids = filtered_Population.loc[~filtered_Population['IDno'].isin(Processed_Problemliste['IDno']), 'IDno']
missing_data = pd.DataFrame({'IDno': missing_ids})
columns_to_add = [col for col in Processed_Problemliste.columns if col != 'IDno']
missing_data[columns_to_add] = 0
Processed_Problemliste = pd.concat([Processed_Problemliste, missing_data], ignore_index=True)
Processed_Problemliste = Processed_Problemliste.groupby('IDno').max().reset_index()
pre_liste.append(Processed_Problemliste)

Process Højde

In [15]:
Processed_Højde = filtered_Højde.drop(columns = "Timestamp")

height_map = {
    'Kvinde': '166,7',
    'Mand': '180,2'
}

merged = pd.merge(Processed_Højde['IDno'], filtered_Population, on='IDno', how='right', indicator=True)

missing_ids = merged[merged['_merge'] == 'right_only']['IDno']

rows_to_append = []
for id_no in missing_ids:
    gender = filtered_Population.loc[filtered_Population['IDno'] == id_no, 'Køn'].iloc[0]
    height_value = height_map[gender]
    rows_to_append.append({'IDno': id_no, 'Højde': height_value})

# Append rows to Processed_Højde
if rows_to_append:
    Processed_Højde = pd.concat([Processed_Højde, pd.DataFrame(rows_to_append)], ignore_index=True)

Processed_Højde['Højde'] = pd.to_numeric(Processed_Højde['Højde'].str.replace(',', '.'), errors='coerce')
Processed_Højde = Processed_Højde.groupby('IDno')['Højde'].mean().reset_index()
pre_liste.append(Processed_Højde)

Process Spirometri

In [17]:
Processed_Spiromitri = filtered_Spiromitri
Processed_Spiromitri['Værdi'] = Processed_Spiromitri['Værdi'].astype(str)
Processed_Spiromitri['Værdi']=Processed_Spiromitri['Værdi'].str.replace('%','')
Processed_Spiromitri['Værdi']=Processed_Spiromitri['Værdi'].str.replace(',','.')
# Iterate over rows and convert 'Værdi' to float if possible, otherwise drop the row
for index, row in Processed_Spiromitri.iterrows():
    value = row['Værdi']
    if value is None:
        continue
    else:
        try:
            float_value = float(value)
            Processed_Spiromitri.at[index, 'Værdi'] = float_value
        except ValueError:
            Processed_Spiromitri.drop(index, inplace=True)

mean_value = Processed_Spiromitri.loc[Processed_Spiromitri['Prøve'] == 'FVC (L)', 'Værdi'].mean()
Processed_Spiromitri.loc[(Processed_Spiromitri['Prøve'] == 'FVC (L)') & (Processed_Spiromitri['Værdi'].isna()), 'Værdi'] = mean_value

pivot_df_spiro = Processed_Spiromitri.pivot_table(index = 'IDno' ,columns='Prøve', values='Værdi', aggfunc='first').reset_index()
df = pivot_df_spiro.rename_axis(None, axis=1)
Processed_Spiromitri = df
# Identify missing IDno entries
missing_ids = filtered_Population.loc[~filtered_Population['IDno'].isin(Processed_Spiromitri['IDno']), 'IDno']

# Create new rows for missing IDno entries with 0 in all other columns
new_rows = pd.DataFrame({'IDno': missing_ids})
for col in Processed_Spiromitri.columns:
    if col != 'IDno':
        new_rows[col] = 0

# Append new rows to Processed_Spiromitri
Processed_Spiromitri = pd.concat([Processed_Spiromitri, new_rows], ignore_index=True)
Processed_Spiromitri = Processed_Spiromitri.drop_duplicates(subset='IDno')
Processed_Spiromitri = Processed_Spiromitri.fillna(0)
pre_liste.append(Processed_Spiromitri)

C:\Users\NSCH0157\AppData\Local\Temp\ipykernel_21712\3481871253.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Processed_Spiromitri = Processed_Spiromitri.fillna(0)


In [18]:
len(Processed_Spiromitri['IDno'].unique())

3185

Process Lab_svar

In [19]:
ja = filtered_Lab_svar_1.drop(columns = 'Timestamp')
grouped = ja.groupby('IDno')
agg_dict = {col: ['mean', 'var'] for col in ja.columns if col != 'IDno'}
result = grouped.agg(agg_dict)
result.columns = ['_'.join(col).strip() for col in result.columns.values]
for col in result.columns:
    if 'var' in col:
        mean_col = col.replace('var', 'mean')
        result[col] = result.apply(lambda row: 0 if pd.notna(row[mean_col]) and pd.isna(row[col]) else row[col], axis=1)
Processed_Lab_svar_1 = result

ja = filtered_Lab_svar_2.drop(columns = 'Timestamp')
grouped = ja.groupby('IDno')
agg_dict = {col: ['mean', 'var'] for col in ja.columns if col != 'IDno'}
result = grouped.agg(agg_dict)
result.columns = ['_'.join(col).strip() for col in result.columns.values]
for col in result.columns:
    if 'var' in col:
        mean_col = col.replace('var', 'mean')
        result[col] = result.apply(lambda row: 0 if pd.notna(row[mean_col]) and pd.isna(row[col]) else row[col], axis=1)
for col in result.columns:
    if 'mean' in col:
        var_col = col.replace('mean', 'var')
        valid_col = col.replace('mean', 'valid')
        result[valid_col] = result.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)
Processed_Lab_svar_2 = result

kol_navn = []
for i in range(len(Processed_Lab_svar_1.columns)):
    if list(Processed_Lab_svar_1.columns)[i] not in kol_navn:
        kol_navn.append(list(Processed_Lab_svar_1.columns)[i])
for i in range(len(Lab_svar_2.columns)):
    if list(Processed_Lab_svar_2.columns)[i] not in kol_navn:
        kol_navn.append(list(Processed_Lab_svar_2.columns)[i])
kol_navn = kol_navn[2:]
ny_list = {}
for i in range(len(kol_navn)):
    tal = 0
    if kol_navn[i] in Processed_Lab_svar_1:
        tal = tal + (len(Processed_Lab_svar_1[~Processed_Lab_svar_1[kol_navn[i]].isna()]))
    if kol_navn[i] in Processed_Lab_svar_2:
        tal = tal + (len(Processed_Lab_svar_2[~Processed_Lab_svar_2[kol_navn[i]].isna()]))
    ny_list[kol_navn[i]] = tal

antal = 0
hvilke = []
for i in range(len(ny_list)):
    if ny_list[kol_navn[i]] >= 1500:
        antal += 1
        hvilke.append(kol_navn[i])
antal = antal / 2

Processed_Lab_svar_1 = Processed_Lab_svar_1[hvilke]
Processed_Lab_svar_2 = Processed_Lab_svar_2[hvilke]

for col in Processed_Lab_svar_1.columns:
    if 'mean' in col:
        var_col = col.replace('mean', 'var')
        valid_col = col.replace('mean', 'valid')
        Processed_Lab_svar_1[valid_col] = Processed_Lab_svar_1.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)
for col in Processed_Lab_svar_2.columns:
    if 'mean' in col:
        var_col = col.replace('mean', 'var')
        valid_col = col.replace('mean', 'valid')
        Processed_Lab_svar_2[valid_col] = Processed_Lab_svar_2.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)

Processed_Lab_svar_1 = Processed_Lab_svar_1.fillna(0)
Processed_Lab_svar_2 = Processed_Lab_svar_2.fillna(0)
Processed_Lab_svar_1.reset_index(inplace = True)
Processed_Lab_svar_2.reset_index(inplace = True)
Processed_Lab_svar = pd.concat([Processed_Lab_svar_1,Processed_Lab_svar_2], ignore_index = True)

Processed_Lab_svar = Processed_Lab_svar.applymap(lambda x: 1 if x == True else (0 if x == False else x))

# Step 2: Add missing IDno from Populationen with all other columns filled with 0
missing_ids = filtered_Population.loc[~filtered_Population['IDno'].isin(Processed_Lab_svar['IDno']), 'IDno']
missing_data = pd.DataFrame({'IDno': missing_ids})
columns_to_add = [col for col in Processed_Lab_svar.columns if col != 'IDno']
missing_data[columns_to_add] = 0

Processed_Lab_svar = pd.concat([Processed_Lab_svar, missing_data], ignore_index=True)
Processed_Lab_svar = Processed_Lab_svar.groupby('IDno').max().reset_index()
pre_liste.append(Processed_Lab_svar)

C:\Users\NSCH0157\AppData\Local\Temp\ipykernel_21712\3302831987.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[valid_col] = result.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)
C:\Users\NSCH0157\AppData\Local\Temp\ipykernel_21712\3302831987.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[valid_col] = result.apply(lambda row: pd.notna(row[col]) and pd.notna(row[var_col]), axis=1)
C:\Users\NSCH0157\AppData\Local\Temp\ipykernel_21712\3302831987.py:25: PerformanceWarning: DataFrame

In [20]:
len(Processed_Lab_svar['IDno'].unique())

3185

Process Dialyse

In [21]:
Processed_Dialyse = filtered_Dialyse.drop(columns =['Blodflow - indstille','Mål for væsketræk', 'Timestamp']) #Vi sletter mål for væsketræk, da det kun er målt 61 gange
encoded_dia1 = pd.get_dummies(filtered_Dialyse['Filtersæt'])
encoded_dia2 = pd.get_dummies(filtered_Dialyse['Væsketype dialysat'])
encoded_dia3 = pd.get_dummies(filtered_Dialyse['CRRT modus'])
df = pd.concat([Processed_Dialyse, encoded_dia1.astype(int), encoded_dia2.astype(int), encoded_dia3.astype(int)], axis=1)
df = df.drop(columns=['Filtersæt','Væsketype dialysat', 'CRRT modus'])
Processed_Dialyse = df
Processed_Dialyse['Valg af antikoagulant til Multifiltrate: CVVHD eller CVVHDF'] = Processed_Dialyse['Valg af antikoagulant til Multifiltrate: CVVHD eller CVVHDF'].apply(lambda x: 1 if pd.notna(x) else 0)
# List of columns to calculate statistics for
columns_of_interest = [
    'Dialysatvæske hastighed - indstillet',
    'Væskefjernelse fra patienten - indstillet',
    'Væskefjernelse fra patienten - aflæst',
    'Væsketræk siden sidste aflæsning (I/U)'
]

# Calculate mean and variance for each "IDno" and column
mean_df = Processed_Dialyse.groupby('IDno')[columns_of_interest].mean().add_suffix('_mean')
var_df = Processed_Dialyse.groupby('IDno')[columns_of_interest].var().add_suffix('_var')

# Merge the mean and variance dataframes with the original dataframe
Processed_Dialyse = Processed_Dialyse.merge(mean_df, on='IDno', how='left')
Processed_Dialyse = Processed_Dialyse.merge(var_df, on='IDno', how='left')
Processed_Dialyse = Processed_Dialyse.drop(columns = columns_of_interest)
Processed_Dialyse = Processed_Dialyse.groupby('IDno').max().reset_index()

# Identify IDno values that are in Populationen but not in Processed_Dialyse
missing_idnos = filtered_Population[~filtered_Population['IDno'].isin(Processed_Dialyse['IDno'])]['IDno']

# Create a dataframe with these missing IDnos and 0 in every other input
missing_data = pd.DataFrame(missing_idnos, columns=['IDno'])
for col in Processed_Dialyse.columns:
    if col != 'IDno':
        missing_data[col] = 0

# Append this missing data to Processed_Dialyse
Processed_Dialyse = pd.concat([Processed_Dialyse, missing_data], ignore_index=True)

# Replace NaN values with 0
Processed_Dialyse.fillna(0, inplace=True)
Processed_Dialyse = Processed_Dialyse.groupby('IDno').max().reset_index()
pre_liste.append(Processed_Dialyse)

In [22]:
len(Processed_Dialyse['IDno'].unique())

3185

Process AoR

In [23]:
Processed_AoR = filtered_AoR
Processed_AoR = Processed_AoR.fillna(0)
# Mapping dictionary
rygning_mapping = {
    'Storryger': 9,
    'Hver dag': 8,
    'Lejlighedsvis ryger': 7,
    'Nogle dage': 6,
    'Ryger, aktuel status ukendt': 5,
    'Tidligere': 4,
    'Udsat for passiv rygning - aldrig været ryger': 3,
    'Aldrig': 2,
    'Ukendt': 1,
    'Aldrig vurderet': 0
}

# Apply mapping to the 'Rygning' column
Processed_AoR['Rygning'] = Processed_AoR['Rygning'].map(rygning_mapping).fillna(Processed_AoR['Rygning'])
pre_liste.append(Processed_AoR)

In [24]:
len(Processed_AoR['IDno'].unique())

3185

Process Populationen - er delt i Processed_Populationen_x og Processed_Populationen_y

In [25]:
Processed_Populationen = filtered_Population.copy()
Processed_Populationen['Operation SKS kode'] = Processed_Populationen['Operation SKS kode'].str[:3]
Processed_Populationen = Processed_Populationen.drop_duplicates(subset=['Operation SKS kode', 'IDno'])
encoded_sks = pd.get_dummies(Processed_Populationen['Operation SKS kode'])
encoded_sks = encoded_sks.astype(int)
# Concatenate the encoded SKS columns with the original DataFrame
df = pd.concat([filtered_Population, encoded_sks], axis=1)
df = df.drop(columns=['Indlæggelsestidspunkt', 'Operationsnavn', 'Udskrivelsestidspunkt', 'Operation SKS kode','Timestamp','Dødsdato','Død inden for 1 år af operation'])
df_grouped = df.groupby('IDno').max()
df_grouped.reset_index(inplace = True)
def set_dead_columns(row):
    if row['dead_days'] == 0:
        row['dead30'] = 0
        row['dead90'] = 0
        row['dead365'] = 0
    elif row['dead_days'] == 1:
        row['dead30'] = 1
        row['dead90'] = 0
        row['dead365'] = 0
    elif row['dead_days'] == 2:
        row['dead30'] = 0
        row['dead90'] = 1
        row['dead365'] = 0
    elif row['dead_days'] == 3:
        row['dead30'] = 0
        row['dead90'] = 0
        row['dead365'] = 1
    return row

df_grouped = df_grouped.apply(set_dead_columns, axis=1)
df_grouped = df_grouped.drop(columns=['dead_days'])
Processed_Populationen = df_grouped
Processed_Populationen_x = Processed_Populationen.drop(columns=['dead30','dead90','dead365'])
Processed_Populationen_y = Processed_Populationen[['IDno','dead30','dead90','dead365']]
pre_liste.append(Processed_Populationen_x)

In [26]:
len(Processed_Populationen['IDno'].unique())

3185

Process KAG_data

In [ ]:
pivot_df_KAG = filtered_KAG_data.pivot_table(index = 'IDno' ,columns='kar lokalisation', values='stenosis', aggfunc='first').reset_index()
df = pivot_df_KAG.rename_axis(None, axis=1)
df = df.dropna(axis=1, how='all')
df = df.dropna(axis=0, how='all')
df.fillna(0, inplace=True)
Processed_KAG_data = df
# Identify missing IDno entries
missing_ids = filtered_Population.loc[~filtered_Population['IDno'].isin(Processed_KAG_data['IDno']), 'IDno']

# Create new rows for missing IDno entries with 0 in all other columns
new_rows = pd.DataFrame({'IDno': missing_ids})
for col in Processed_KAG_data.columns:
    if col != 'IDno':
        new_rows[col] = 0

# Append new rows to Processed_Spiromitri
Processed_KAG_data = pd.concat([Processed_KAG_data, new_rows], ignore_index=True)
Processed_KAG_data = Processed_KAG_data.groupby('IDno').max().reset_index()
Processed_KAG_data = Processed_KAG_data.fillna(0)
pre_liste.append(Processed_KAG_data)

In [28]:
len(Processed_KAG_data['IDno'].unique())

3185

Process Ekkokardiografi

In [ ]:
pivot_df_Ekko = filtered_Ekkokardiografi.pivot_table(index = 'IDno' ,columns='Komponentnavn', values='Resultatværdi', aggfunc='first').reset_index()
df = pivot_df_Ekko.rename_axis(None, axis=1)
df = df.dropna(axis=1, how='all')
Processed_Ekkokardiografi = df.dropna(axis=0, how='all')
for col in Processed_Ekkokardiografi.columns:
    if col != 'IDno':
        # Create a new column with 1 if there is an input, and 0 if it's NaN
        new_col = col + '_has_input'
        Processed_Ekkokardiografi[new_col] = Processed_Ekkokardiografi[col].notna().astype(int)
        
        # Replace NaN values with 0 in the original column
        Processed_Ekkokardiografi[col].fillna(0, inplace=True)
# Identify missing IDno entries
missing_ids = filtered_Population.loc[~filtered_Population['IDno'].isin(Processed_Ekkokardiografi['IDno']), 'IDno']

# Create new rows for missing IDno entries with 0 in all other columns
new_rows = pd.DataFrame({'IDno': missing_ids})
for col in Processed_Ekkokardiografi.columns:
    if col != 'IDno':
        new_rows[col] = 0

# Append new rows to Processed_Spiromitri
Processed_Ekkokardiografi = pd.concat([Processed_Ekkokardiografi, new_rows], ignore_index=True)
Processed_Ekkokardiografi = Processed_Ekkokardiografi.groupby('IDno').max().reset_index()
Processed_Ekkokardiografi = Processed_Ekkokardiografi.fillna(0)
pre_liste.append(Processed_Ekkokardiografi)

In [30]:
len(Processed_Ekkokardiografi['IDno'].unique())

3185

Process OE1O

In [ ]:
Processed_OE1O = filtered_OE1O.copy()
Processed_OE1O['Operation SKS kode'] = Processed_OE1O['Operation SKS kode'].str[:3]
Processed_OE1O = Processed_OE1O.drop_duplicates(subset=['Operation SKS kode', 'IDno'])
encoded_sks = pd.get_dummies(Processed_OE1O['Operation SKS kode'])
# Concatenate the encoded SKS columns with the original DataFrame
df = pd.concat([filtered_OE1O, encoded_sks], axis=1)
df = df.drop(columns = 'Operation SKS kode')
df = df.drop(columns = 'Operationsnavn')
df = df.drop(columns = 'Timestamp')
cols_to_convert = df.columns.difference(['IDno'])

# Convert True/False to 1/0 and replace NaN with 0 for all columns except 'IDno'
df[cols_to_convert] = df[cols_to_convert].applymap(lambda x: 1 if x is True else (0 if x is False or pd.isna(x) else x))
Processed_OE1O = df.groupby('IDno').max().reset_index()
# pre_liste.append(Processed_OE1O)

In [32]:
len(Processed_OE1O['IDno'].unique())

3185

Process EKG_data

In [33]:
dropliste_EKG = filtered_EKG_data[filtered_EKG_data['Navn'] != 'Autofortolkning']
dropliste_EKG = dropliste_EKG[dropliste_EKG['Navn'] != 'Kommentarer fra EKG-Apparat']
pivot_df_EKG = dropliste_EKG.pivot_table(index = 'IDno' ,columns='Navn', values='Værdi', aggfunc='first').reset_index()
Processed_EKG_Data = pivot_df_EKG.rename_axis(None, axis=1)
# Identify missing IDno entries
missing_ids = filtered_Population.loc[~filtered_Population['IDno'].isin(Processed_EKG_Data['IDno']), 'IDno']

# Create new rows for missing IDno entries with 0 in all other columns
new_rows = pd.DataFrame({'IDno': missing_ids})
for col in Processed_EKG_Data.columns:
    if col != 'IDno':
        new_rows[col] = 0

# Append new rows to Processed_Spiromitri
Processed_EKG_Data = pd.concat([Processed_EKG_Data, new_rows], ignore_index=True)
Processed_EKG_Data = Processed_EKG_Data.groupby('IDno').max().reset_index()
Processed_EKG_Data = Processed_EKG_Data.fillna(0)
pre_liste.append(Processed_EKG_Data)

In [34]:
len(Processed_EKG_Data['IDno'].unique())

3185

In [35]:
len(Processed_OE1O['IDno']), len(Processed_Dialyse['IDno']), len(Processed_EKG_Data['IDno']), len(Processed_Ekkokardiografi['IDno']), len(Processed_Højde['IDno']), len(Processed_KAG_data['IDno']), len(Processed_Lab_svar['IDno']), len(Processed_Populationen['IDno']), len(Processed_Problemliste['IDno']), len(Processed_Spiromitri['IDno']), len(Processed_Vægt['IDno'])

(3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185)

In [36]:
len(Processed_OE1O['IDno'].unique()), len(Processed_Dialyse['IDno'].unique()), len(Processed_EKG_Data['IDno'].unique()), len(Processed_Ekkokardiografi['IDno'].unique()), len(Processed_Højde['IDno'].unique()), len(Processed_KAG_data['IDno'].unique()), len(Processed_Lab_svar['IDno'].unique()), len(Processed_Populationen['IDno'].unique()), len(Processed_Problemliste['IDno'].unique()), len(Processed_Spiromitri['IDno'].unique()), len(Processed_Vægt['IDno'].unique())

(3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185)

In [37]:
len(Processed_OE1O.columns)+ len(Processed_Dialyse.columns)+ len(Processed_EKG_Data.columns)+ len(Processed_Ekkokardiografi.columns)+ len(Processed_Højde.columns)+ len(Processed_KAG_data.columns)+ len(Processed_Lab_svar.columns)+ len(Processed_Populationen.columns)+ len(Processed_Problemliste.columns)+ len(Processed_Spiromitri.columns)+ len(Processed_Vægt.columns)

586

X-matrice tid

In [ ]:
from functools import reduce
dataframes_to_merge = [Processed_OE1O, Processed_Dialyse, Processed_EKG_Data, Processed_Ekkokardiografi, Processed_Højde, Processed_KAG_data, Processed_Lab_svar, Processed_Populationen, Processed_Problemliste, Processed_Spiromitri, Processed_Vægt]
x_matrix_pre = reduce(lambda left, right: pd.merge(left, right, on='IDno', how='outer'), dataframes_to_merge)
x_matrix_pre

In [39]:
x_matrix_pre.to_csv('x_matrix_pre_3185.csv',compression='gzip',index=False)

In [ ]:
pd.read_csv('x_matrix_pre_3185.csv', compression='gzip')